In [35]:
import pandas as pd
import geopandas as gpd
import datetime
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))

path = 'data/trips.json'
trips = gpd.read_file(path)
trips['Trip'] = trips['SlideOrder']

In [71]:
def get_time(date):
    date = str(date)[0:10]
    date = pd.Timestamp(int(date), unit='s').strftime('%#m-%#d-%Y')
    return date

def line_agg(group):
    x_list = []
    y_list = []
    [[
        (x_list.append(x), y_list.append(y)) for (x,y) in line.coords
    ] for line in group]

    x_min, x_max = min(x_list), max(x_list)
    y_min, y_max = min(y_list), max(y_list)

    bounds = [[x_min, y_min], [x_max, y_max]]

    return bounds

def first_date(dates):
    date = list(dates)[0]
    date = get_time(date)

    return date

bounds = trips.groupby('Trip').agg(
    TripName = ('TripName', 'first'),
    StartDate = ('Start_Date', first_date),
    EndDate = ('End_Date', first_date),
    Era = ('Era', 'first'),
    bounds = ('geometry', line_agg)
    ).reset_index(drop=False)

bounds['EraName'] = bounds['TripName'].apply(lambda s: s.split(':')[0])
equal_filt = bounds['EraName']!=bounds['TripName']
bounds.loc[equal_filt, 'TripName'] = bounds.loc[equal_filt, 'TripName'].apply(lambda s: s.split(':')[1])
bounds.loc[bounds['EraName']==bounds['TripName'], 'TripName'] = ''

bounds['content'] = ''


In [73]:
bounds.to_json('data/slides.json', orient='records')